In [1]:
import cv2
import numpy as np
import copy
import math
import os
import time

In [2]:
def get_background():
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    #Opencv集成了BackgroundSubtractorMOG2用于动态目标检测，用到的是基于自适应混合高斯背景建模的背景减除法。
    isBgCaptured = 1
    print('!!!Background Captured!!!')
    return bgModel,isBgCaptured
    
def removeBG(frame): #移除背景
    fgmask = bgModel.apply(frame, learningRate=learningRate) #计算前景掩膜
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1) #使用特定的结构元素来侵蚀图像。
    res = cv2.bitwise_and(frame, frame, mask=fgmask) #使用掩膜移除静态背景
    return res

def reset_background():
    bgModel = None
    isBgCaptured[0] = 0
    print('!!!Reset BackGround!!!')
    return bgModel,isBgCaptured
    
def finger_detection(frame,bgModel,isBgCaptured):
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # 双边滤波
    frame = cv2.flip(frame, 1)  # 翻转  0:沿X轴翻转(垂直翻转)   大于0:沿Y轴翻转(水平翻转)   小于0:先沿X轴翻转，再沿Y轴翻转，等价于旋转180°
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),(frame.shape[1], int(cap_region_y_end * frame.shape[0])), (0, 0, 255), 2)
    #画矩形框  frame.shape[0]表示frame的高度    frame.shape[1]表示frame的宽度   注：opencv的像素是BGR顺序
    cv2.imshow('original', frame)   #经过双边滤波后的初始化窗口
    if isBgCaptured == 1:  # isBgCaptured[0] == 1 表示已经捕获背景
        img = removeBG(frame)  #移除背景
        img = img[0:int(cap_region_y_end * frame.shape[0]),int(cap_region_x_begin * frame.shape[1]):frame.shape[1]]  # 剪切右上角矩形框区域
        cv2.imshow('mask', img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #将移除背景后的图像转换为灰度图
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)  #加高斯模糊
        cv2.imshow('blur', blur)
        ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)  #二值化处理
        cv2.imshow('binary', thresh)
        
    # get the coutours
        thresh1 = copy.deepcopy(thresh)
        contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #寻找轮廓   注：这里的'_'用作变量名称，_表示一个变量被指定了名称，但不打算使用。
        length = len(contours)
        maxArea = -1
        if length > 0:
            for i in range(length):  # 找到最大的轮廓（根据面积）
                temp = contours[i]
                area = cv2.contourArea(temp)  #计算轮廓区域面积
                if area > maxArea:
                    maxArea = area
                    ci = i

            res = contours[ci]  #得出最大的轮廓区域
            hull = cv2.convexHull(res)  #得出点集（组成轮廓的点）的凸包
            drawing = np.zeros(img.shape, np.uint8)
            cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)   #画出最大区域轮廓
            cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)  #画出凸包轮廓

            moments = cv2.moments(res)  # 求最大区域轮廓的各阶矩
            if moments['m00']==0 or moments['m00']==0:
                return 0
            center = (int(moments['m10'] / moments['m00']), int(moments['m01'] / moments['m00']))
            cv2.circle(drawing, center, 8, (0,0,255), -1)   #画出重心

            fingerRes = []   #寻找指尖
            max = 0; count = 0; notice = 0; cnt = 0;allmax=0
            for i in range(len(res)):
                temp = res[i]
                dist = (temp[0][0] -center[0])*(temp[0][0] -center[0]) + (temp[0][1] -center[1])*(temp[0][1] -center[1]) #计算重心到轮廓边缘的距离
                
                if dist > max:
                    max = dist
                    notice = i
                if dist > allmax:
                    allmax = dist
                if dist != max:
                    count = count + 1
                    if count > 40:
                        count = 0
                        max = 0
                        flag = False   #布尔值
                        if center[1] < res[notice][0][1]:   #低于手心的点不算
                            continue
                        for j in range(len(fingerRes)):  #离得太近的不算
                            if abs(res[notice][0][0]-fingerRes[j][0]) < 70:
                                flag = True
                                break
                        #if ((res[notice][0][0] -center[0])*(res[notice][0][0] -center[0])+(res[notice][0][1] -center[1])*(res[notice][0][1] -center[1]))<0.2*allmax*allmax:
                           # flag = True
                           # break    
                        if flag :
                            continue
                        fingerRes.append(res[notice][0])
                        #print(fingerRes)
                        cv2.circle(drawing, tuple(res[notice][0]), 8 , (255, 0, 0), -1) #画出指尖
                        cv2.line(drawing, center, tuple(res[notice][0]), (255, 0, 0), 2)
                        cnt = cnt + 1
            cv2.imshow('output', drawing)
            print("fingers num =",cnt)
            return(cnt)
    else:
        return 0      

def style_transfer(model,
                    pathIn='',
                   pathOut='',
                   width=None,
                   jpg_quality=80,
                    ):
    ## 从本地加载预训练模型
    print('加载预训练模型......')
    print(pathIn)
    ## 读入原始图片，调整图片至所需尺寸，然后获取图片的宽度和高度
    img = cv2.imread(pathIn)
    (h, w) = img.shape[:2]
    if width is not None:
        img = cv2.resize(img, (width, round(width*h/w)), interpolation=cv2.INTER_CUBIC)
        (h, w) = img.shape[:2]

    ## 将图片构建成一个blob：设置图片尺寸，将各通道像素值减去平均值（比如ImageNet所有训练样本各通道统计平均值）
    ## 然后执行一次前馈网络计算，并输出计算所需的时间
    blob = cv2.dnn.blobFromImage(img, 1.0, (w, h), (103.939, 116.779, 123.680), swapRB=False, crop=False)
    model.setInput(blob)
    start = time.time()
    output = model.forward()
    end = time.time()
    print("风格迁移花费：{:.2f}秒".format(end - start))

    ## reshape输出结果, 将减去的平均值加回来，并交换各颜色通道
    output = output.reshape((3, output.shape[2], output.shape[3]))
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.680
    output = output.transpose(1, 2, 0)
    
    ## 输出风格化后的图片
    cv2.imwrite(pathOut, output, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_quality])   
    


In [1]:
isBgCaptured = 0  # 布尔类型, 背景是否被捕获
threshold =70
bgModel=None
# 参数
cap_region_x_begin = 0.5  # 起点/总宽度
cap_region_y_end = 0.8
threshold = 70  # 二值化阈值
blurValue = 41  # 高斯模糊参数
bgSubThreshold = 50
learningRate = 0
model_num=0
# 相机/摄像头
camera = cv2.VideoCapture(0)   #打开电脑自带摄像头，如果参数是1会打开外接摄像头
camera1 = cv2.VideoCapture(1)
camera.set(10, 200)   #设置视频属性
paths=model_path()
while camera.isOpened():
    
    ret, frame = camera.read()
    finger_num=finger_detection(frame,bgModel,isBgCaptured)
    if finger_num==1:
        model_num=(model_num+1)%7
    model_path=paths[model_num]
    model = cv2.dnn.readNetFromTorch(model_path)
    
    # 输入的键盘值
    k = cv2.waitKey(10)
    if k == 27:  # 按下ESC退出
        break
    elif k == ord('b'):  # 按下'b'会捕获背景
        bgModel,isBgCaptured = get_background()
    elif k == ord('r'):  # 按下'r'会重置背景
        bgModel,isBgCaptured = reset_background()


camera.release()
camera1.release()
cv2.destroyAllWindows() 

NameError: name 'cv2' is not defined

['candy.t7', 'feathers.t7', 'la_muse.t7', 'mosaic.t7', 'starry_night.t7', 'the_scream.t7', 'udnie.t7'] ['D:\\jupyter\\homework\\model/candy.t7', 'D:\\jupyter\\homework\\model/feathers.t7', 'D:\\jupyter\\homework\\model/la_muse.t7', 'D:\\jupyter\\homework\\model/mosaic.t7', 'D:\\jupyter\\homework\\model/starry_night.t7', 'D:\\jupyter\\homework\\model/the_scream.t7', 'D:\\jupyter\\homework\\model/udnie.t7']


In [111]:
cv2.imshow('mask', img)

djsk


In [43]:
# 变量
isBgCaptured = 0  # 布尔类型, 背景是否被捕获
triggerSwitch = False  # 如果正确，键盘模拟器将工作

# 相机/摄像头
camera = cv2.VideoCapture(0)   #打开电脑自带摄像头，如果参数是1会打开外接摄像头
camera.set(10, 200)   #设置视频属性
cv2.namedWindow('trackbar') #设置窗口名字
cv2.resizeWindow("trackbar", 640, 200)  #重新设置窗口尺寸
cv2.createTrackbar('threshold', 'trackbar', threshold, 100, printThreshold)
#createTrackbar是Opencv中的API，其可在显示图像的窗口中快速创建一个滑动控件，用于手动调节阈值，具有非常直观的效果。

while camera.isOpened():
    ret, frame = camera.read()
    threshold = cv2.getTrackbarPos('threshold', 'trackbar') #返回滑动条上的位置的值（即实时更新阈值）
    # frame = cv2.cvtColor(frame,cv2.COLOR_RGB2YCrCb)
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # 双边滤波
    frame = cv2.flip(frame, 1)  # 翻转  0:沿X轴翻转(垂直翻转)   大于0:沿Y轴翻转(水平翻转)   小于0:先沿X轴翻转，再沿Y轴翻转，等价于旋转180°
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),(frame.shape[1], int(cap_region_y_end * frame.shape[0])), (0, 0, 255), 2)
    #画矩形框  frame.shape[0]表示frame的高度    frame.shape[1]表示frame的宽度   注：opencv的像素是BGR顺序
    cv2.imshow('original', frame)   #经过双边滤波后的初始化窗口

    #主要操作
    if isBgCaptured == 1:  # isBgCaptured == 1 表示已经捕获背景
        img = removeBG(frame)  #移除背景
        img = img[0:int(cap_region_y_end * frame.shape[0]),int(cap_region_x_begin * frame.shape[1]):frame.shape[1]]  # 剪切右上角矩形框区域
        cv2.imshow('mask', img)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #将移除背景后的图像转换为灰度图
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)  #加高斯模糊
        cv2.imshow('blur', blur)
        ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)  #二值化处理
        cv2.imshow('binary', thresh)

        # get the coutours
        thresh1 = copy.deepcopy(thresh)
        contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #寻找轮廓   注：这里的'_'用作变量名称，_表示一个变量被指定了名称，但不打算使用。
        length = len(contours)
        maxArea = -1
        if length > 0:
            for i in range(length):  # 找到最大的轮廓（根据面积）
                temp = contours[i]
                area = cv2.contourArea(temp)  #计算轮廓区域面积
                if area > maxArea:
                    maxArea = area
                    ci = i

            res = contours[ci]  #得出最大的轮廓区域
            #print(res)
            hull = cv2.convexHull(res)  #得出点集（组成轮廓的点）的凸包
            print(hull)
            drawing = np.zeros(img.shape, np.uint8)
            cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)   #画出最大区域轮廓
            cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)  #画出凸包轮廓

            moments = cv2.moments(res)  # 求最大区域轮廓的各阶矩
            center = (int(moments['m10'] / moments['m00']), int(moments['m01'] / moments['m00']))
            cv2.circle(drawing, center, 8, (0,0,255), -1)   #画出重心

            fingerRes = []   #寻找指尖
            max = 0; count = 0; notice = 0; cnt = 0
            for i in range(len(res)):
                temp = res[i]
                dist = (temp[0][0] -center[0])*(temp[0][0] -center[0]) + (temp[0][1] -center[1])*(temp[0][1] -center[1]) #计算重心到轮廓边缘的距离
                if dist > max:
                    max = dist
                    notice = i
                if dist != max:
                    count = count + 1
                    if count > 40:
                        count = 0
                        max = 0
                        flag = False   #布尔值
                        if center[1] < res[notice][0][1]:   #低于手心的点不算
                            continue
                        for j in range(len(fingerRes)):  #离得太近的不算
                            if abs(res[notice][0][0]-fingerRes[j][0]) < 30 :
                                flag = True
                                break
                        if flag :
                            continue
                        fingerRes.append(res[notice][0])
                        cv2.circle(drawing, tuple(res[notice][0]), 8 , (255, 0, 0), -1) #画出指尖
                        cv2.line(drawing, center, tuple(res[notice][0]), (255, 0, 0), 2)
                        cnt = cnt + 1
            print(fingerRes)
            cv2.imshow('output', drawing)
            print("fingers num =",cnt)

    # 输入的键盘值
    k = cv2.waitKey(10)
    if k == 27:  # 按下ESC退出
        break
    elif k == ord('b'):  # 按下'b'会捕获背景
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
        #Opencv集成了BackgroundSubtractorMOG2用于动态目标检测，用到的是基于自适应混合高斯背景建模的背景减除法。
        isBgCaptured = 1
        print('!!!Background Captured!!!')
    elif k == ord('r'):  # 按下'r'会重置背景
        bgModel = None
        triggerSwitch = False
        isBgCaptured = 0
        print('!!!Reset BackGround!!!')


camera.release()
cv2.destroyAllWindows() 

!!!Background Captured!!!
[[[319 383]]

 [[141 383]]

 [[140 378]]

 [[140 360]]

 [[161 216]]

 [[168 179]]

 [[176 143]]

 [[177 140]]

 [[236   0]]

 [[319   0]]]
[array([236,   0], dtype=int32), array([296,  15], dtype=int32)]
fingers num = 2
[[[135 378]]

 [[134 383]]

 [[ 79 383]]

 [[ 79 372]]

 [[ 81 366]]

 [[ 82 364]]

 [[ 84 361]]

 [[ 89 356]]

 [[ 95 352]]

 [[ 98 351]]

 [[103 351]]

 [[106 352]]

 [[125 359]]

 [[127 360]]

 [[131 364]]

 [[133 368]]

 [[134 371]]

 [[135 376]]]
[]
fingers num = 0
[[[177 369]]

 [[176 374]]

 [[174 381]]

 [[173 383]]

 [[ 77 383]]

 [[ 73 374]]

 [[ 73 369]]

 [[ 88 324]]

 [[ 90 321]]

 [[ 92 319]]

 [[ 96 317]]

 [[ 99 316]]

 [[104 315]]

 [[113 315]]

 [[128 316]]

 [[130 317]]

 [[133 319]]

 [[168 352]]

 [[174 358]]

 [[176 362]]

 [[177 366]]]
[]
fingers num = 0
[[[169 354]]

 [[168 356]]

 [[166 359]]

 [[162 363]]

 [[159 365]]

 [[156 366]]

 [[153 366]]

 [[ 97 365]]

 [[ 91 364]]

 [[ 85 361]]

 [[ 80 356]]

 [[ 78 352]]

 

[array([ 78, 129], dtype=int32)]
fingers num = 1
[[[161 237]]

 [[160 240]]

 [[158 244]]

 [[156 247]]

 [[153 249]]

 [[151 250]]

 [[148 251]]

 [[ 96 268]]

 [[ 86 268]]

 [[ 82 266]]

 [[ 78 262]]

 [[ 76 258]]

 [[ 70 240]]

 [[ 69 236]]

 [[ 69 230]]

 [[ 72 137]]

 [[ 73 133]]

 [[ 74 131]]

 [[ 77 128]]

 [[ 81 128]]

 [[ 83 130]]

 [[158 222]]

 [[160 226]]

 [[161 229]]]
[array([ 77, 128], dtype=int32)]
fingers num = 1
[[[161 237]]

 [[160 240]]

 [[158 244]]

 [[156 247]]

 [[153 249]]

 [[151 250]]

 [[148 251]]

 [[ 96 268]]

 [[ 86 268]]

 [[ 82 266]]

 [[ 78 262]]

 [[ 76 258]]

 [[ 70 240]]

 [[ 69 236]]

 [[ 69 230]]

 [[ 72 137]]

 [[ 73 133]]

 [[ 74 131]]

 [[ 77 128]]

 [[ 81 128]]

 [[ 83 130]]

 [[158 222]]

 [[160 226]]

 [[161 229]]]
[array([ 77, 128], dtype=int32)]
fingers num = 1
[[[162 235]]

 [[161 239]]

 [[158 245]]

 [[154 249]]

 [[150 251]]

 [[ 97 268]]

 [[ 91 269]]

 [[ 86 268]]

 [[ 84 267]]

 [[ 78 261]]

 [[ 76 257]]

 [[ 75 254]]

 [[ 70 238]]


[array([ 82, 128], dtype=int32)]
fingers num = 1
[[[162 235]]

 [[160 241]]

 [[158 244]]

 [[157 245]]

 [[154 247]]

 [[148 249]]

 [[ 96 266]]

 [[ 86 266]]

 [[ 84 265]]

 [[ 81 263]]

 [[ 79 261]]

 [[ 77 258]]

 [[ 75 254]]

 [[ 73 248]]

 [[ 71 241]]

 [[ 70 237]]

 [[ 70 228]]

 [[ 75 151]]

 [[ 76 138]]

 [[ 77 134]]

 [[ 79 130]]

 [[ 81 128]]

 [[ 84 127]]

 [[ 87 128]]

 [[ 88 129]]

 [[159 220]]

 [[162 226]]]
[array([ 84, 127], dtype=int32)]
fingers num = 1
[[[162 237]]

 [[161 240]]

 [[159 244]]

 [[155 248]]

 [[153 249]]

 [[150 250]]

 [[ 97 266]]

 [[ 92 267]]

 [[ 90 267]]

 [[ 86 266]]

 [[ 82 264]]

 [[ 79 261]]

 [[ 77 258]]

 [[ 75 254]]

 [[ 74 251]]

 [[ 71 241]]

 [[ 70 237]]

 [[ 70 225]]

 [[ 75 146]]

 [[ 76 138]]

 [[ 77 135]]

 [[ 79 131]]

 [[ 80 130]]

 [[ 82 129]]

 [[ 84 129]]

 [[126 171]]

 [[159 220]]

 [[161 223]]

 [[162 226]]]
[array([ 82, 129], dtype=int32)]
fingers num = 1
[[[161 239]]

 [[160 242]]

 [[159 244]]

 [[157 247]]

 [[155 249]]


[array([149, 113], dtype=int32), array([ 75, 126], dtype=int32)]
fingers num = 2
[[[158 237]]

 [[157 241]]

 [[156 244]]

 [[150 250]]

 [[148 251]]

 [[ 99 267]]

 [[ 94 268]]

 [[ 88 268]]

 [[ 85 267]]

 [[ 81 265]]

 [[ 78 262]]

 [[ 76 259]]

 [[ 75 255]]

 [[ 74 233]]

 [[ 71 151]]

 [[ 71 133]]

 [[ 72 130]]

 [[ 73 128]]

 [[ 74 127]]

 [[ 76 126]]

 [[147 113]]

 [[152 113]]

 [[154 114]]

 [[155 115]]

 [[156 118]]

 [[158 233]]]
[array([147, 113], dtype=int32), array([ 76, 126], dtype=int32)]
fingers num = 2
[[[158 237]]

 [[157 241]]

 [[156 244]]

 [[150 250]]

 [[148 251]]

 [[ 99 267]]

 [[ 94 268]]

 [[ 88 268]]

 [[ 85 267]]

 [[ 81 265]]

 [[ 78 262]]

 [[ 76 259]]

 [[ 75 255]]

 [[ 74 233]]

 [[ 71 151]]

 [[ 71 133]]

 [[ 72 130]]

 [[ 73 128]]

 [[ 74 127]]

 [[ 76 126]]

 [[147 113]]

 [[152 113]]

 [[154 114]]

 [[155 115]]

 [[156 118]]

 [[158 233]]]
[array([147, 113], dtype=int32), array([ 76, 126], dtype=int32)]
fingers num = 2
[[[157 240]]

 [[156 243]]

 

[array([143, 113], dtype=int32), array([ 98, 120], dtype=int32), array([ 44, 197], dtype=int32), array([212, 191], dtype=int32)]
fingers num = 4
[[[212 194]]

 [[165 248]]

 [[160 253]]

 [[158 254]]

 [[101 276]]

 [[ 98 277]]

 [[ 89 277]]

 [[ 85 275]]

 [[ 79 269]]

 [[ 53 221]]

 [[ 49 213]]

 [[ 43 200]]

 [[ 42 197]]

 [[ 42 195]]

 [[ 96 123]]

 [[ 98 122]]

 [[143 113]]

 [[147 113]]

 [[149 114]]

 [[183 133]]

 [[185 135]]

 [[212 192]]]
[array([143, 113], dtype=int32), array([ 98, 122], dtype=int32), array([ 42, 195], dtype=int32), array([212, 192], dtype=int32)]
fingers num = 4
[[[212 194]]

 [[163 251]]

 [[160 253]]

 [[158 254]]

 [[101 276]]

 [[ 98 277]]

 [[ 93 278]]

 [[ 92 278]]

 [[ 88 277]]

 [[ 86 276]]

 [[ 81 271]]

 [[ 79 268]]

 [[ 49 216]]

 [[ 45 208]]

 [[ 44 205]]

 [[ 44 195]]

 [[ 95 122]]

 [[ 96 121]]

 [[ 98 120]]

 [[145 112]]

 [[148 113]]

 [[183 133]]

 [[184 134]]

 [[185 136]]

 [[212 191]]]
[array([145, 112], dtype=int32), array([ 98, 120], d

...

[[[200 192]]

 [[198 195]]

 [[160 242]]

 [[154 248]]

 [[148 251]]

 [[104 272]]

 [[ 98 274]]

 [[ 89 274]]

 [[ 87 273]]

 [[ 81 267]]

 [[ 79 263]]

 [[ 77 257]]

 [[ 76 252]]

 [[ 74 239]]

 [[ 74 230]]

 [[ 77 128]]

 [[ 78 124]]

 [[ 80 122]]

 [[ 82 121]]

 [[127 110]]

 [[130 110]]

 [[132 111]]

 [[164 128]]

 [[166 130]]

 [[200 188]]]
[array([127, 110], dtype=int32), array([ 80, 122], dtype=int32), array([200, 188], dtype=int32)]
fingers num = 3
[[[200 191]]

 [[199 193]]

 [[159 244]]

 [[153 250]]

 [[149 252]]

 [[103 272]]

 [[100 273]]

 [[ 96 274]]

 [[ 92 274]]

 [[ 89 273]]

 [[ 85 271]]

 [[ 82 268]]

 [[ 80 265]]

 [[ 78 260]]

 [[ 75 245]]

 [[ 74 239]]

 [[ 74 229]]

 [[ 79 125]]

 [[ 80 123]]

 [[ 81 122]]

 [[127 111]]

 [[131 111]]

 [[165 128]]

 [[166 129]]

 [[200 189]]]
[array([127, 111], dtype=int32), array([ 81, 122], dtype=int32), array([200, 189], dtype=int32)]
fingers num = 3
[[[200 191]]

 [[198 194]]

 [[159 245]]

 [[156 248]]

 [[152 250]]

 [[1

[array([128, 111], dtype=int32), array([ 82, 122], dtype=int32), array([200, 187], dtype=int32)]
fingers num = 3
[[[201 189]]

 [[199 193]]

 [[159 243]]

 [[153 249]]

 [[101 272]]

 [[ 98 273]]

 [[ 89 273]]

 [[ 86 272]]

 [[ 83 270]]

 [[ 79 266]]

 [[ 77 263]]

 [[ 76 261]]

 [[ 68 240]]

 [[ 67 237]]

 [[ 67 232]]

 [[ 78 127]]

 [[ 79 124]]

 [[ 80 122]]

 [[ 82 121]]

 [[128 111]]

 [[133 111]]

 [[165 129]]

 [[166 130]]

 [[201 187]]]
[array([128, 111], dtype=int32), array([ 82, 121], dtype=int32), array([201, 187], dtype=int32)]
fingers num = 3
[[[201 189]]

 [[200 191]]

 [[198 194]]

 [[158 245]]

 [[155 248]]

 [[153 249]]

 [[ 99 273]]

 [[ 94 274]]

 [[ 90 274]]

 [[ 87 273]]

 [[ 85 272]]

 [[ 82 270]]

 [[ 80 268]]

 [[ 78 265]]

 [[ 76 261]]

 [[ 67 240]]

 [[ 66 237]]

 [[ 66 231]]

 [[ 79 126]]

 [[ 80 124]]

 [[ 82 122]]

 [[130 110]]

 [[132 110]]

 [[134 111]]

 [[165 129]]

 [[166 130]]

 [[201 186]]]
[array([130, 110], dtype=int32), array([ 82, 122], dtype=int

[array([111, 111], dtype=int32), array([180, 179], dtype=int32)]
fingers num = 2
[[[181 182]]

 [[180 184]]

 [[151 236]]

 [[147 240]]

 [[106 262]]

 [[100 265]]

 [[ 94 267]]

 [[ 87 267]]

 [[ 84 266]]

 [[ 81 264]]

 [[ 79 262]]

 [[ 77 259]]

 [[ 76 257]]

 [[ 75 254]]

 [[ 74 250]]

 [[ 71 235]]

 [[ 71 223]]

 [[ 72 200]]

 [[ 73 196]]

 [[105 118]]

 [[106 116]]

 [[108 113]]

 [[112 111]]

 [[114 111]]

 [[138 123]]

 [[140 125]]

 [[181 179]]]
[array([112, 111], dtype=int32), array([181, 179], dtype=int32)]
fingers num = 2
[[[181 180]]

 [[180 183]]

 [[151 237]]

 [[148 240]]

 [[105 263]]

 [[ 99 266]]

 [[ 95 267]]

 [[ 86 267]]

 [[ 84 266]]

 [[ 81 264]]

 [[ 80 263]]

 [[ 78 260]]

 [[ 75 254]]

 [[ 74 251]]

 [[ 72 243]]

 [[ 71 237]]

 [[ 71 223]]

 [[ 72 200]]

 [[ 73 196]]

 [[106 116]]

 [[107 114]]

 [[109 112]]

 [[111 111]]

 [[115 111]]

 [[139 123]]

 [[181 179]]]
[array([111, 111], dtype=int32), array([181, 179], dtype=int32)]
fingers num = 2
[[[181 182]]

 

[[[193 189]]

 [[192 191]]

 [[157 240]]

 [[153 244]]

 [[147 247]]

 [[100 269]]

 [[ 94 271]]

 [[ 86 271]]

 [[ 82 269]]

 [[ 78 265]]

 [[ 35 206]]

 [[ 33 203]]

 [[ 31 199]]

 [[ 31 196]]

 [[ 78 123]]

 [[ 80 121]]

 [[120 113]]

 [[124 113]]

 [[126 114]]

 [[155 130]]

 [[156 131]]

 [[193 186]]]
[array([120, 113], dtype=int32), array([ 80, 121], dtype=int32), array([ 31, 196], dtype=int32), array([193, 186], dtype=int32)]
fingers num = 4
[[[191 191]]

 [[189 195]]

 [[158 239]]

 [[152 245]]

 [[148 247]]

 [[100 269]]

 [[ 94 271]]

 [[ 85 271]]

 [[ 83 270]]

 [[ 78 265]]

 [[ 36 207]]

 [[ 34 204]]

 [[ 32 200]]

 [[ 32 197]]

 [[ 78 124]]

 [[ 79 123]]

 [[ 81 122]]

 [[120 113]]

 [[124 113]]

 [[126 114]]

 [[155 130]]

 [[156 131]]

 [[191 187]]]
[array([120, 113], dtype=int32), array([ 81, 122], dtype=int32), array([ 32, 197], dtype=int32), array([191, 187], dtype=int32)]
fingers num = 4
[[[193 188]]

 [[192 191]]

 [[190 194]]

 [[156 241]]

 [[151 246]]

 [[149 247

[[[190 188]]

 [[189 190]]

 [[153 240]]

 [[149 244]]

 [[143 247]]

 [[ 96 270]]

 [[ 93 271]]

 [[ 85 271]]

 [[ 81 269]]

 [[ 78 266]]

 [[ 36 207]]

 [[ 34 204]]

 [[ 32 200]]

 [[ 32 195]]

 [[ 74 126]]

 [[ 76 124]]

 [[117 113]]

 [[121 113]]

 [[151 129]]

 [[152 130]]

 [[190 184]]]
[array([117, 113], dtype=int32), array([ 76, 124], dtype=int32), array([ 32, 195], dtype=int32), array([190, 184], dtype=int32)]
fingers num = 4
